<a href="https://colab.research.google.com/github/hankedwards/AppleWatch/blob/master/AppleHealth2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analyze Apple Health Data

## Import the libraries

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime
from datetime import datetime, timedelta
import numpy as np

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

## Create element tree object

In [ ]:
tree =ET.parse('/content/drive/MyDrive/Colab Notebooks/Apple Health Data/export.xml')

In [ ]:
# for every health record, extract the attributes into roo
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

In [ ]:
data = pd.DataFrame(record_list)

## Enable Google Colab Dataframe formatter

In [ ]:
from google.colab import data_table
# from vega_datasets import data

data_table.enable_dataframe_formatter()

In [ ]:
# proper type to dates
for  col in ['creationDate', 'startDate', 'endDate']:
  data[col] = pd.to_datetime(data[col])

In [ ]:
data.info()

In [ ]:
# value is numeric, NaN if fails
data['value'] =pd.to_numeric(data['value'], errors = 'coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time )makes it easier to aggregate
data['value'] = data['value'].fillna(1.0)

In [ ]:
data['type'] = data['type'].str.replace('HKQuantityTypeIdentifier', '')
data['type'] = data['type'].str.replace('HKCategoryTypeIdentifier', '')

In [ ]:
# Show distribution of type records
data['type'].value_counts()

## Drop unneeded columns

In [ ]:
data.drop(['sourceName', 'sourceVersion','device'], axis=1, inplace = True)

In [ ]:
# Show the datafram information

print(data.info())

In [ ]:
data.set_index('creationDate', inplace =True)

### Now get only the swimming records

In [ ]:
swim_df = (data[data.index >= '2018-08-22']) 
swim_df1 = swim_df[swim_df['type'] == 'DistanceSwimming']
precovid = (swim_df1[swim_df1.index <= '2020-02-21']) 
postcovid = (swim_df1[swim_df1.index >= '2021-06-01'])
precovid['covid'] = 'Pre'
postcovid['covid'] = 'Post'

swim_df2 = precovid
# precovid.info()
precovid.head(4)
# postcovid.info()
# print(swim_df.info())
# print(swim_df1.info())
swim_df2 = pd.concat([precovid, postcovid],  ignore_index=True,)

In [ ]:
swim_df2.info()

In [ ]:
# datetime.date(2020, 3, 1)


In [ ]:
# covid_datedt = pd.to_datetime('2020-03-01')
# coviddt = datetime.date(2020, 3, 1)
# # covid = datetime.date(2020, 3, 1)
# # covid_datedt = covid_date.to_datetime
# # covid_datedt = pd.to_datetime(covid_date)
# # dt_obj = datetime.fromtimestamp(covid_datedt)
# covid_datedt
# coviddt

In [ ]:
# swim_df1['startDate'] >= covid_date

##Now set a variable to indicate whether the time was before covid or after covic so we can determine if there is a difference in the average time

In [ ]:
# create new colvid for covid indicator
# swim_df1['covid'] = False

###Write function to set covid variable based on date

In [ ]:
# def covid_grp(x):
#   if x <= ('2022-03-01'):
#     return 'True'
#   elif x > ('2022-03-01'): 
#     return 'False' 

# swim_df1['covid'] = swim_df1['creationDate'].apply(covid_grp)   

df.loc[df['Age'] < 30, 'Age Category'] = 'Under 30'

###df.loc[df.ID == 103, 'FirstName'] = "Matt"

#now identify erroneous rows

In [ ]:
# swim_df1.loc[swim_df1.index >= '2021-08-22'] 

#Identify Covid dates

In [ ]:
# swim_df1.loc[swim_df1['startDate'] <= pd.datetime('2020, 03,01'), 'covid'] == 'False'
# swim_df1.loc[swim_df1['startDate'] >= d, 'covid'] == 'True'
# swim_df1.loc[swim_df1['startDate'] >= covid_datedt, 'covid'] == True

In [ ]:
swim_df2['covid'].value_counts()
# swim_df1.info()
# swim_df2.tail(5)

#Drop the offending rows

In [ ]:
# Create string column for the start date for further summarization and grouping
swim_df2['start'] = pd.to_datetime(swim_df2['startDate']).dt.date


# Drop the rows where the length of the lap was 16, as this must have been
# when I swam at the clubhouse
# drop records where the yards/lap = 16
swim_df2.drop(swim_df2[swim_df2['value'] == 16].index, inplace = True)


In [ ]:
# swim_df1['start'] = pd.to_datetime(swim_df1['startDate']).dt.date
swim_df2.info()

##Convert the timedelta to second to get seconds per lap

In [ ]:
swim_df2['secperlap'] = (swim_df2['endDate'] - swim_df2['startDate'])/ np.timedelta64(1, 's')
swim_df2.info()
print(swim_df2.head(5))

### Now get statistics on seconds per lap

In [ ]:
#swim_df1[["secperlap"]].describe()

In [ ]:
swim_by_date = swim_df2.groupby(["covid", "start"])["secperlap"].mean()
print(swim_by_date)

In [ ]:
covidmean = swim_df2.groupby(["covid"])["secperlap"].mean()

In [ ]:
#swim_by_date.info
#swim_by_date['secperlab'].describe


In [ ]:
swim_by_datedf = swim_by_date.to_frame().reset_index()
print(swim_by_datedf.info())

In [ ]:
print(swim_by_datedf.info())
swim_by_datedf

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=swim_by_datedf['start'],y=swim_by_datedf['secperlap']))
fig.update_layout(title_text='<b>Average Time per Lap</b>', title_x=0.5)
fig.show()

In [ ]:
import plotly.express as px
# df = px.swim_df1.secperlap()s
fig = px.histogram(swim_df2, x="secperlap",
                   marginal="violin") # or violin, rug
                  #  hover_data=df.columns)
fig.show()

In [ ]:
# import plotly.express as px
# df = px.data.tips()
fig = px.scatter(swim_by_datedf, x="start", y='secperlap')
fig.show()

##Now segregate the data between befor covid and after covid

In [ ]:
import plotly.express as px
# df = px.swim_by_datedf.secperlap()
fig = px.box(swim_by_datedf, x="covid", y="secperlap", points='all',notched=True)
fig.update_traces(quartilemethod="inclusive")
fig.show()

In [ ]:
fig = sns.FacetGrid(swim_by_datedf,hue='covid', aspect=4, size =10)
fig.map(sns.kdeplot,'secperlap',shade=True)
#oldest = titanic_df['Age'].max()
#fig.set(xlim=(0,oldest))
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(swim_by_datedf,hue='covid', aspect=3, size=10)
fig.map(sns.kdeplot,'secperlap',shade=True)
#oldest = titanic_df['Age'].max()
#fig.set(xlim=(0,oldest))
fig.add_legend()

In [ ]:
# sns.set(rc={"figure.figsize":(6, 5)}) #width=6, height=5
ax = sns.boxplot(x='covid', y='secperlap', data=swim_by_datedf, color='#99c2a2')
ax = sns.swarmplot(x='covid', y='secperlap', data=swim_by_datedf, color='#7d0013')

# plt.show()